In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json as simplejson

In [114]:
data = []
with open("yelp_academic_dataset_business.json") as f:
    for line in f:
        data.append(simplejson.loads(line))
df = pd.DataFrame(data[:5000])                                  
df.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,open,review_count,stars,state,type
0,"{'Drive-Thru': False, 'Good For Groups': True,...",5UmKMjUEUNdYWqANhGckJw,"[Fast Food, Restaurants]",Dravosburg,"4734 Lebanon Church Rd\nDravosburg, PA 15034","{'Monday': {'close': '21:00', 'open': '11:00'}...",40.354327,-79.900706,Mr Hoagie,[],True,7,3.5,PA,business
1,"{'Happy Hour': True, 'Price Range': 1, 'Good F...",UsFtqoBl7naz8AVUBZMjQQ,[Nightlife],Dravosburg,"202 McClure St\nDravosburg, PA 15034",{},40.350553,-79.886814,Clancy's Pub,[],True,5,3.0,PA,business
2,{'Good for Kids': True},cE27W9VPgO88Qxe4ol6y_g,"[Active Life, Mini Golf, Golf]",Bethel Park,"1530 Hamilton Rd\nBethel Park, PA 15234",{},40.354115,-80.014660,Cool Springs Golf Center,[],False,5,2.5,PA,business
3,"{'Good For Groups': True, 'Take-out': True, 'P...",mVHrayjG3uZ_RLHkLj-AMg,"[Bars, American (New), Nightlife, Lounges, Res...",Braddock,"414 Hawkins Ave\nBraddock, PA 15104","{'Thursday': {'close': '19:00', 'open': '10:00...",40.408830,-79.866211,Emil's Lounge,[],True,26,4.5,PA,business
4,"{'Good for Kids': True, 'Good For Groups': Tru...",mYSpR_SLPgUVymYOvTQd_Q,"[Active Life, Golf]",Braddock,"1000 Clubhouse Dr\nBraddock, PA 15104","{'Thursday': {'close': '20:00', 'open': '11:00...",40.403405,-79.855782,Grand View Golf Club,[],True,3,5.0,PA,business


Create a new column that contains only the zipcode.

In [115]:
#df.full_address[0].split()[-1]
df["zipcode"] = df.apply(lambda x: x['full_address'].split(' ')[-1],1)

The table contains a column called 'categories' and each entry in this column is populated by a list. We are interested in those businesses that are restaurants. Create a new column 'restaurant_type' that contains a description of the restaurant based on the other elements of 'categories. That is, if we have '[Sushi Bars, Japanese, Restaurants]' in categories the 'restaurant_type will be '{'SushiBars': 1, 'Japanese': 1, 'Mexican': 0, ...}'

In [116]:
df.categories[:10]

0                             [Fast Food, Restaurants]
1                                          [Nightlife]
2                       [Active Life, Mini Golf, Golf]
3    [Bars, American (New), Nightlife, Lounges, Res...
4                                  [Active Life, Golf]
5    [Bars, American (Traditional), Nightlife, Rest...
6                     [Auto Repair, Automotive, Tires]
7                             [Active Life, Mini Golf]
8    [Roofing, Home Services, Decks & Railing, Cont...
9                                [Veterinarians, Pets]
Name: categories, dtype: object

In [117]:
categories = {}

for index, row in df.iterrows():
    if "Restaurants" in row['categories']:        
          for el in row['categories']:
            if el != "Restaurants": 
                categories[el]=0 
list(categories)[:10]


['Hot Dogs',
 'Italian',
 'Breakfast & Brunch',
 'Hawaiian',
 'Cheesesteaks',
 'Steakhouses',
 'Health Markets',
 'Pakistani',
 'Comfort Food',
 'Food']

In [118]:
def category(row):

    d=dict(categories)
    if "Restaurants" in row['categories']:        
        for el in row['categories']:
            if el in d.keys(): 
                d[el] = 1
            else:
                d[el]=0 
    else:             
         d= "Remove"
   
    return d

In [119]:
df['restaurant_type'] = df.apply(lambda x: category(x), 1)


Lets clean the 'attributes' column. The entries in this column are dictionaries. We need to do two things:

Turn all the True or False values in the dictionary to 1 and 0.

There are some entries within dictionaries that are dictionaries themselves, lets turn the whole entry into just one dictionary, for example if we have

'{'Accepts Credit Cards': True, 'Alcohol': 'none','Ambience': {'casual': False,'classy': False}}'

then turn it into

'{'Accepts Credit Cards':1, 'Alcohol': 0, 'Ambience_casual': 0, 'Ambience_classy': 0}'.

There might be other entries like {'Price Range': 1} where the values are numerical so we might want to change that into {'Price_Range_1': 1}.

In [120]:
def flatten(row):
    d=row['attributes']
    d1 ={}
    for el in d.keys():

        if type (d[el]) == bool:
            d1[el] = int(d[el])
        elif type (d[el]) == str:            
            label = '{}_{}'.format(el, d[el])
            d1[label] = 1
        elif type (d[el]) == int:           
            label = '{}_{}'.format(el, d[el])            
            d1[label] = 1
        elif type (d[el]) == dict:                   
                for elem in d[el].keys():
                    label = '{}_{}'.format(el, elem)            
                    d1[label] = int(d[el][elem])
        else:         
             print ([el], d[el])            
    return d1               

In [121]:
df['attributes_cleaned'] = df.apply(lambda x: flatten(x), 1)

Create a new column for every day of the week and fill it with the amount of hours the business is open that day.

In [123]:
days =['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']               

In [129]:
df.hours[0]

{'Friday': {'close': '21:00', 'open': '11:00'},
 'Monday': {'close': '21:00', 'open': '11:00'},
 'Thursday': {'close': '21:00', 'open': '11:00'},
 'Tuesday': {'close': '21:00', 'open': '11:00'},
 'Wednesday': {'close': '21:00', 'open': '11:00'}}

In [141]:
FMT ="%H:%M"
def time(row,day):

    try:
        closing = datetime.strptime(row['hours'][day]['close'], FMT)
        opening = datetime.strptime(row['hours'][day]['open'], FMT)

        time = closing - opening

        if time <= timedelta(days=0):
            time+= timedelta(days=1)
    except: 
        time=timedelta(days=0)
        
    return time    
        

In [142]:
for day in days:
    df[day] = df.apply(lambda x: time(x, day), 1)

In [143]:
df.head()

,attributes,business_id,categories,city,full_address,hours,latitude,longitude,name,neighborhoods,...,zipcode,restaurant_type,attributes_cleaned,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,"{'Drive-Thru': False, 'Good For Groups': True,...",5UmKMjUEUNdYWqANhGckJw,"[Fast Food, Restaurants]",Dravosburg,"4734 Lebanon Church Rd\nDravosburg, PA 15034","{'Monday': {'close': '21:00', 'open': '11:00'}...",40.354327,-79.900706,Mr Hoagie,[],...,15034,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...","{'Parking_validated': 0, 'Good For Groups': 1,...",10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,00:00:00,00:00:00
1,"{'Happy Hour': True, 'Price Range': 1, 'Good F...",UsFtqoBl7naz8AVUBZMjQQ,[Nightlife],Dravosburg,"202 McClure St\nDravosburg, PA 15034",{},40.350553,-79.886814,Clancy's Pub,[],...,15034,Remove,"{'Happy Hour': 1, 'Good For Groups': 1, 'Price...",00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00
2,{'Good for Kids': True},cE27W9VPgO88Qxe4ol6y_g,"[Active Life, Mini Golf, Golf]",Bethel Park,"1530 Hamilton Rd\nBethel Park, PA 15234",{},40.354115,-80.014660,Cool Springs Golf Center,[],...,15234,Remove,{'Good for Kids': 1},00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00
3,"{'Good For Groups': True, 'Take-out': True, 'P...",mVHrayjG3uZ_RLHkLj-AMg,"[Bars, American (New), Nightlife, Lounges, Res...",Braddock,"414 Hawkins Ave\nBraddock, PA 15104","{'Thursday': {'close': '19:00', 'open': '10:00...",40.408830,-79.866211,Emil's Lounge,[],...,15104,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...","{'Parking_validated': 0, 'Good For Groups': 1,...",00:00:00,09:00:00,09:00:00,09:00:00,10:00:00,06:00:00,00:00:00
4,"{'Good for Kids': True, 'Good For Groups': Tru...",mYSpR_SLPgUVymYOvTQd_Q,"[Active Life, Golf]",Braddock,"1000 Clubhouse Dr\nBraddock, PA 15104","{'Thursday': {'close': '20:00', 'open': '11:00...",40.403405,-79.855782,Grand View Golf Club,[],...,15104,Remove,"{'Parking_valet': 0, 'Parking_validated': 0, '...",00:00:00,00:00:00,09:00:00,09:00:00,09:00:00,09:00:00,05:00:00


Create a table with the average star review for a business from the review table.



In [144]:
data_review = []
i = 0
with open("yelp_academic_dataset_review.json") as f:
    for line in f:
        i+=1
        data_review.append(simplejson.loads(line))
        if i == 5000:
            break

df_review = pd.DataFrame(data_review[:])                                  
df_review.head()

,business_id,date,review_id,stars,text,type,user_id,votes
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,Ya85v4eqdd6k9Od8HbQjyA,4,"Mr Hoagie is an institution. Walking in, it do...",review,PUFPaY9KxDAcGqfsorJp3Q,"{'cool': 0, 'useful': 0, 'funny': 0}"
1,5UmKMjUEUNdYWqANhGckJw,2014-02-13,KPvLNJ21_4wbYNctrOwWdQ,5,Excellent food. Superb customer service. I mis...,review,Iu6AxdBYGR4A0wspR9BYHA,"{'cool': 0, 'useful': 0, 'funny': 0}"
2,5UmKMjUEUNdYWqANhGckJw,2015-10-31,fFSoGV46Yxuwbr3fHNuZig,5,Yes this place is a little out dated and not o...,review,auESFwWvW42h6alXgFxAXQ,"{'cool': 0, 'useful': 1, 'funny': 1}"
3,5UmKMjUEUNdYWqANhGckJw,2015-12-26,pVMIt0a_QsKtuDfWVfSk2A,3,PROS: Italian hoagie was delicious. Friendly ...,review,qiczib2fO_1VBG8IoCGvVg,"{'cool': 0, 'useful': 0, 'funny': 0}"
4,5UmKMjUEUNdYWqANhGckJw,2016-04-08,AEyiQ_Y44isJmNbMTyoMKQ,2,First the only reason this place could possibl...,review,qEE5EvV-f-s7yHC0Z4ydJQ,"{'cool': 0, 'useful': 1, 'funny': 0}"


In [145]:
average_rating= df_review.groupby('business_id')['stars'].mean().reset_index()
average_rating.columns = ['business_id', 'star_avg']
df_review = df_review.merge(average_rating, how = 'left', on = 'business_id')
df_review.head()

,business_id,date,review_id,stars,text,type,user_id,votes,star_avg
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,Ya85v4eqdd6k9Od8HbQjyA,4,"Mr Hoagie is an institution. Walking in, it do...",review,PUFPaY9KxDAcGqfsorJp3Q,"{'cool': 0, 'useful': 0, 'funny': 0}",3.428571
1,5UmKMjUEUNdYWqANhGckJw,2014-02-13,KPvLNJ21_4wbYNctrOwWdQ,5,Excellent food. Superb customer service. I mis...,review,Iu6AxdBYGR4A0wspR9BYHA,"{'cool': 0, 'useful': 0, 'funny': 0}",3.428571
2,5UmKMjUEUNdYWqANhGckJw,2015-10-31,fFSoGV46Yxuwbr3fHNuZig,5,Yes this place is a little out dated and not o...,review,auESFwWvW42h6alXgFxAXQ,"{'cool': 0, 'useful': 1, 'funny': 1}",3.428571
3,5UmKMjUEUNdYWqANhGckJw,2015-12-26,pVMIt0a_QsKtuDfWVfSk2A,3,PROS: Italian hoagie was delicious. Friendly ...,review,qiczib2fO_1VBG8IoCGvVg,"{'cool': 0, 'useful': 0, 'funny': 0}",3.428571
4,5UmKMjUEUNdYWqANhGckJw,2016-04-08,AEyiQ_Y44isJmNbMTyoMKQ,2,First the only reason this place could possibl...,review,qEE5EvV-f-s7yHC0Z4ydJQ,"{'cool': 0, 'useful': 1, 'funny': 0}",3.428571


Create a new table that only contains restaurants with the following schema: Business_Name | Restaurant_type | Friday hours | Saturday hours | Attributes | Zipcode | Average Rating

In [146]:
df_final = df_review.merge(df, how = 'left', on = 'business_id')
df_final.head()
#df.head()

,business_id,date,review_id,stars_x,text,type_x,user_id,votes,star_avg,attributes,...,zipcode,restaurant_type,attributes_cleaned,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,5UmKMjUEUNdYWqANhGckJw,2012-08-01,Ya85v4eqdd6k9Od8HbQjyA,4,"Mr Hoagie is an institution. Walking in, it do...",review,PUFPaY9KxDAcGqfsorJp3Q,"{'cool': 0, 'useful': 0, 'funny': 0}",3.428571,"{'Drive-Thru': False, 'Good For Groups': True,...",...,15034,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...","{'Parking_validated': 0, 'Good For Groups': 1,...",10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,0 days,0 days
1,5UmKMjUEUNdYWqANhGckJw,2014-02-13,KPvLNJ21_4wbYNctrOwWdQ,5,Excellent food. Superb customer service. I mis...,review,Iu6AxdBYGR4A0wspR9BYHA,"{'cool': 0, 'useful': 0, 'funny': 0}",3.428571,"{'Drive-Thru': False, 'Good For Groups': True,...",...,15034,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...","{'Parking_validated': 0, 'Good For Groups': 1,...",10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,0 days,0 days
2,5UmKMjUEUNdYWqANhGckJw,2015-10-31,fFSoGV46Yxuwbr3fHNuZig,5,Yes this place is a little out dated and not o...,review,auESFwWvW42h6alXgFxAXQ,"{'cool': 0, 'useful': 1, 'funny': 1}",3.428571,"{'Drive-Thru': False, 'Good For Groups': True,...",...,15034,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...","{'Parking_validated': 0, 'Good For Groups': 1,...",10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,0 days,0 days
3,5UmKMjUEUNdYWqANhGckJw,2015-12-26,pVMIt0a_QsKtuDfWVfSk2A,3,PROS: Italian hoagie was delicious. Friendly ...,review,qiczib2fO_1VBG8IoCGvVg,"{'cool': 0, 'useful': 0, 'funny': 0}",3.428571,"{'Drive-Thru': False, 'Good For Groups': True,...",...,15034,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...","{'Parking_validated': 0, 'Good For Groups': 1,...",10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,0 days,0 days
4,5UmKMjUEUNdYWqANhGckJw,2016-04-08,AEyiQ_Y44isJmNbMTyoMKQ,2,First the only reason this place could possibl...,review,qEE5EvV-f-s7yHC0Z4ydJQ,"{'cool': 0, 'useful': 1, 'funny': 0}",3.428571,"{'Drive-Thru': False, 'Good For Groups': True,...",...,15034,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...","{'Parking_validated': 0, 'Good For Groups': 1,...",10:00:00,10:00:00,10:00:00,10:00:00,10:00:00,0 days,0 days


In [147]:
df_final = df_final[df_final["restaurant_type"] != 'Remove']

In [149]:
df_final = df_final[["name","restaurant_type", "Friday","Saturday","attributes_cleaned","zipcode","star_avg"]]

In [150]:
df_final.head()

,name,restaurant_type,Friday,Saturday,attributes_cleaned,zipcode,star_avg
0,Mr Hoagie,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...",10:00:00,0 days,"{'Parking_validated': 0, 'Good For Groups': 1,...",15034,3.428571
1,Mr Hoagie,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...",10:00:00,0 days,"{'Parking_validated': 0, 'Good For Groups': 1,...",15034,3.428571
2,Mr Hoagie,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...",10:00:00,0 days,"{'Parking_validated': 0, 'Good For Groups': 1,...",15034,3.428571
3,Mr Hoagie,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...",10:00:00,0 days,"{'Parking_validated': 0, 'Good For Groups': 1,...",15034,3.428571
4,Mr Hoagie,"{'Hot Dogs': 0, 'Tapas Bars': 0, 'Gas & Servic...",10:00:00,0 days,"{'Parking_validated': 0, 'Good For Groups': 1,...",15034,3.428571
